# boundary semantic detection


In [1]:
import os
import numpy as np
import tensorflow as tf
import cv2
from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from glob import glob
# import Augmentor
from dataloader import DataLoader
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.data import AUTOTUNE

from torchvision import transforms
caffe_root = r'C:\Users\alext\Jupyter\caffe' # this file is expected to be in {caffe_root}/examples/hed/

from modules import caffe
import sys
sys.path.insert(0, caffe_root + 'python')


### Data Path 

In [ ]:
path = r'C:\Users\alext\Desktop\dataset\AHP'

# original =sorted(glob(os.path.join(path, 'AHP3k1/*')))
# mask = sorted(glob(os.path.join(path, 'AHP3kMasking1/*')))
original_path = r'C:\Users\alext\Desktop\dataset\AHP\AHP3k1'
mask_path = r'C:\Users\alext\Desktop\dataset\AHP\AHP3kMasking1'
edge_path = r'C:\Users\alext\Desktop\dataset\AHP\AHP3kHED1'

ori = [os.path.join(original_path,x) for x in os.listdir(original_path)]
mask = [os.path.join(mask_path,x) for x in os.listdir(mask_path)]
edge = [os.path.join(edge_path,x) for x in os.listdir(edge_path)]

len(mask)


### Preprocessing Data

In [ ]:
CLASS_NUM = 1
SMOOTH = 1e-15
INIT_LR = 1e-2
#TEST_SPLIT = 0.05
VAL_SPLIT = 0.2
IMAGE_SIZE = 224
EPOCHS = 2
batch_size = 32


In [ ]:
# Parse the images and masks, and return the data in batches, augmented optionally.

dataset = DataLoader(image_paths=ori,
                     mask_paths=mask,
                     image_size=(255, 255),
                     crop_percent=0.8,
                     channels=(3, 1),
                     augment=True,
                     compose=False,
                     seed=47)

dataset = dataset.data_batch(batch_size=32,
                             shuffle=True)

In [ ]:
TOTAL_SIZE = len(ori)
VALID_SIZE = int(VAL_SPLIT * TOTAL_SIZE)


In [ ]:
#split IMAGES data into train and validation set
train_x, valid_x = train_test_split(ori, test_size=VALID_SIZE, random_state=42)

#split MASKS data into train and validation set
train_y, valid_y = train_test_split(mask, test_size=VALID_SIZE, random_state=42)

train_z, valid_z = train_test_split(edge, test_size=VALID_SIZE, random_state = 42)

In [ ]:
len(train_x)

In [ ]:
# train = ImageDataGenerator(rescale = 1/255)
# valid = ImageDataGenerator(rescale = 1/255)

In [ ]:
# train_dataset = train.flow_from_directory(r'C:\Users\alext\Desktop\dataset\AHP\AHP3k1')

### Model

In [ ]:
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Conv3D
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD
import pyimage
from modules.keras_models import MobileNetV3SmallSegmentation
from tensorflow.image import ResizeMethod


In [ ]:

def load_images(imagePath, labelPath):
    # read the image from disk, decode it, convert the data type to
    # floating point, and resize it
    image = tf.io.read_file(imagePath)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.image.resize(image, (224, 224))

    #image = tf.image.per_image_standardization(image)
    
    label = tf.io.read_file(labelPath)
    label = tf.image.decode_png(label, channels=3)
    label = tf.image.rgb_to_grayscale(label)
    label = tf.image.convert_image_dtype(label, dtype=tf.float32)
    #label = tf.image.per_image_standardization(label)
    label = tf.image.resize(label, (224, 224))

    # return the image and the label
    return (image, label)

def augment_using_ops(images, labels):
    images = tf.image.random_brightness(images, 0.2)
    images = tf.image.random_contrast(images, 0.5, 2.0)
    images = tf.image.random_saturation(images, 0.75, 1.25)
    images = tf.image.random_hue(images, 0.1)
    return (images, labels)

In [ ]:
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + SMOOTH) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + SMOOTH)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)


In [ ]:
def IoU_loss(y_true, y_pred):

    numerator = tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred) - numerator 

    jac =  numerator / (denominator + tf.keras.backend.epsilon())

    return 1 - jac

def IoU(y_true, y_pred, t=0.5):


    y_pred_ = tf.cast(y_pred > t, dtype=tf.int32)
    y_true = tf.cast(y_true, dtype=tf.int32)

    TP = tf.math.count_nonzero(y_pred_ * y_true)
    FP = tf.math.count_nonzero(y_pred_ * (y_true - 1))
    FN = tf.math.count_nonzero((y_pred_ - 1) * y_true)

    jac = tf.cond(tf.greater((TP + FP + FN), 0), lambda: TP / (TP + FP + FN),
                  lambda: tf.cast(0.000, dtype='float64'))

    return jac

In [ ]:
img_rows = 224
img_cols = 224

input_shape = (img_rows, img_cols, 3)


In [ ]:
# model = Sequential()
# # model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
# model.add(tf.keras.layers.Reshape((224, 224, 3), input_shape=(224, 224, 3)))

# model.add(Conv2D(32, (3, 3), activation='relu', padding='same',  input_shape=(batch_size,224, 224, 3),data_format='channels_first'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# # model.add(Conv2D(32, (3, 3), activation='relu'))
# # model.add(MaxPooling2D(pool_size=(2, 2)))

# # model.add(Conv2D(32, (3, 3), activation='relu'))
# # model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(256, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(512, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(512, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# # model.add(Conv2D(64, (3, 3), activation='relu'))
# # model.add(Conv2D(64, (3, 3), activation='relu'))
# # model.add(Conv2D(64, (3, 3), activation='relu'))

# model.add(Flatten())
# model.add(Dense(1024, input_shape=(3072,), activation="sigmoid"))
# model.add(tf.keras.layers.Dense(512, activation="sigmoid"))
# # model.add(Dense(128,  input_shape=(255),activation='softmax'))

# # print(model.summary())


In [ ]:

model = MobileNetV3SmallSegmentation(alpha=1.0, shape=(224, 224), n_class=CLASS_NUM,
                 avg_pool_kernel=(11, 11), avg_pool_strides=(4, 4),
                 resize_method=ResizeMethod.BILINEAR, backbone='small')

In [ ]:

opt = Adam(lr=INIT_LR, clipvalue = 5.0)
#opt = Adam(lr=INIT_LR, clip_norm = 1.0)

#m = tf.keras.metrics.MeanIoU(CLASS_NUM)
metrics = [dice_coef,"accuracy", IoU]
#metrics = [dice_coef, "accuracy"]

model.compile(loss=dice_loss, optimizer=opt, metrics=metrics)


In [ ]:
train_steps = len(train_x)//batch_size
valid_steps = len(valid_x)//batch_size

if len(train_x) % batch_size != 0:
    train_steps += 1
    
if len(valid_x) % batch_size != 0:
    valid_steps += 1

In [ ]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

# build the training dataset and data input pipeline
trainDS = tf.data.Dataset.from_tensor_slices((ori,mask))

#trainDS = train_ds.map(lambda x, y: (normalization_layer(x), y))
trainDS = (trainDS
    .map(load_images, num_parallel_calls=AUTOTUNE)
    .cache()
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

#trainDS = trainDS.map(lambda x, y: (normalization_layer(x), y))
#image_batch, labels_batch = next(iter(normalized_ds))

# build the validation dataset and data input pipeline
valDS = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))
valDS = (valDS
    .map(load_images, num_parallel_calls=AUTOTUNE)
    .cache()
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)
#valDS = valDS.map(lambda x, y: (normalization_layer(x), y))

# build the testing dataset and data input pipeline
#testDS = tf.data.Dataset.from_tensor_slices((test_x, test_y))
#testDS = (testDS
    #.map(load_images, num_parallel_calls=AUTOTUNE)
    #.cache()
    #.batch(BATCH)
    #.prefetch(AUTOTUNE)
#)

In [ ]:
bagging = BaggingClassifier(KNeighborsClassifier(),
                             max_samples=0.5, max_features=0.5)

In [ ]:
H = model.fit(
    trainDS,
    validation_data=valDS.repeat(),
    epochs=100,
    steps_per_epoch=len(train_x)//batch_size,
    validation_steps=len(valid_x)//batch_size,
)

### HED

In [ ]:
import argparse
import pyimage
import cv2
from os.path import dirname, join


In [ ]:
class CropLayer(object):
    def __init__(self, params, blobs):
        # initialize our starting and ending (x, y)-coordinates of
        # the crop
        self.startX = 0
        self.startY = 0
        self.endX = 0
        self.endY = 0
        
    def getMemoryShapes(self, inputs):
        # the crop layer will receive two inputs -- we need to crop
        # the first input blob to match the shape of the second one,
        # keeping the batch size and number of channels
        (inputShape, targetShape) = (inputs[0], inputs[1])
        (batchSize, numChannels) = (inputShape[0], inputShape[1])
        (H, W) = (targetShape[2], targetShape[3])
        # compute the starting and ending crop coordinates
        self.startX = int((inputShape[3] - targetShape[3]) / 2)
        self.startY = int((inputShape[2] - targetShape[2]) / 2)
        self.endX = self.startX + W
        self.endY = self.startY + H
        # return the shape of the volume (we'll perform the actual
        # crop during the forward pass
        return [[batchSize, numChannels, H, W]]
    
    def forward(self, inputs):
    # use the derived (x, y)-coordinates to perform the crop
        return [inputs[0][:, :, self.startY:self.endY, self.startX:self.endX]]

In [ ]:
# load our serialized edge detector from disk
print("[INFO] loading edge detector...")

protoPath = join(dirname(r" C:\Users\alext\Jupyter\modules\hed-edge-detector-master\edge_detector"),
                 r"C:\Users\alext\Jupyter\modules\hed-edge-detector-master\deploy.prototxt")

modelPath = r"modules\hed-edge-detector-master\hed_pretrained_bsds.caffemodel"

net = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
# register our new layer with the model
cv2.dnn_registerLayer("Crop", CropLayer)

### Detection

In [ ]:
# load the input image and grab its dimensions
for items in 
image = cv2.imread(r'C:\Users\alext\Desktop\1kAHP\Background\image1.PNG')
(H, W) = image.shape[:2]

In [ ]:
# construct a blob out of the input image for the Holistically-Nested
# Edge Detector
blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size=(W, H),
mean=(104.00698793, 116.66876762, 122.67891434),
swapRB=False, crop=False)

# set the blob as the input to the network and perform a forward pass
# to compute the edges
print("[INFO] performing holistically-nested edge detection...")
net.setInput(blob)
hed = net.forward()
hed = cv2.resize(hed[0, 0], (W, H))
hed = (255 * hed).astype("uint8")

In [ ]:
canny = cv2.Canny(image,100,200)


In [ ]:
cv2.imshow("image", image)
cv2.imshow("HED", hed)
cv2.imshow("canny", canny)

# cv2.imwrite(r"C:\Users\alext\Desktop\1kAHP\hed2.jpg",hed)
# cv2.imwrite(r"C:\Users\alext\Desktop\1kAHP\canny.jpg",canny)

cv2.waitKey(0)
cv2.destroyAllWindows()

### Building

In [ ]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

# build the training dataset and data input pipeline
trainDS = tf.data.Dataset.from_tensor_slices((train_x, train_y))
#trainDS = train_ds.map(lambda x, y: (normalization_layer(x), y))
trainDS = (trainDS
    .map(load_images, num_parallel_calls=AUTOTUNE)
    .cache()
    .batch(BATCH)
    .prefetch(AUTOTUNE)
)

#trainDS = trainDS.map(lambda x, y: (normalization_layer(x), y))
#image_batch, labels_batch = next(iter(normalized_ds))

# build the validation dataset and data input pipeline
valDS = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))
valDS = (valDS
    .map(load_images, num_parallel_calls=AUTOTUNE)
    .cache()
    .batch(BATCH)
    .prefetch(AUTOTUNE)
)
#valDS = valDS.map(lambda x, y: (normalization_layer(x), y))

# build the testing dataset and data input pipeline
#testDS = tf.data.Dataset.from_tensor_slices((test_x, test_y))
#testDS = (testDS
    #.map(load_images, num_parallel_calls=AUTOTUNE)
    #.cache()
    #.batch(BATCH)
    #.prefetch(AUTOTUNE)
#)



### Augmentor

In [ ]:
p = Augmentor.Pipeline(r'C:\Users\alext\Desktop\dataset\AHP\AHP3k1')


# Add operations to the pipeline as normal:
p.rotate(probability=1, max_left_rotation=5, max_right_rotation=5)
p.flip_left_right(probability=0.5)
p.zoom_random(probability=0.5, percentage_area=0.8)
p.flip_top_bottom(probability=0.5)


In [ ]:
# we create two instances with the same arguments
data_gen_args = dict(featurewise_center=True,
                     featurewise_std_normalization=True,
                     rotation_range=90,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2)

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

## STACKING MODEL

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import StackingRegressor



In [ ]:
def get_dataset():
    X, y = make_regression(n_samples=1000, n_features=20, n_informative=15, noise=0.1, random_state=1)
    return X, y


In [ ]:
# get a stacking ensemble of models
def get_stacking():
    # define the base models
    level0 = list()
    level0.append(('knn', KNeighborsRegressor()))
    level0.append(('cart', DecisionTreeRegressor()))
    level0.append(('svm', SVR()))
    # define meta learner model
    level1 = LinearRegression()
    # define the stacking ensemble
    model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
    return model

In [ ]:
# get a list of models to evaluate
def get_models():
    models = dict()
    models['knn'] = KNeighborsRegressor()
    models['cart'] = DecisionTreeRegressor()
    models['svm'] = SVR()
    models['stacking'] = get_stacking()
    return models


In [ ]:
# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
    return scores

In [ ]:
# define dataset
train_z, train_z = get_dataset()
# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


clf = RandomForestClassifier(max_depth=None, min_samples_split=2,
    random_state=0)
clf.fit(train_y,train_z)

